<a href="https://colab.research.google.com/github/ergul13/mr_akgul/blob/main/Medel_RESNET_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# --- 1. ORTAM KURULUMU VE BAĞLANTI ---
from google.colab import drive
import pandas as pd
import numpy as np
import os
import torch

# Drive Bağla
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# Cihaz Kontrolü (GPU Şart)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz: {device}")
if device.type != 'cuda':
    print("UYARI: GPU aktif değil! Eğitim çok yavaş olur. Çalışma Zamanı > Türü Değiştir > T4 GPU seç.")

# --- 2. VERİ SETİNİ YÜKLE ---
csv_path = '/content/drive/MyDrive/FINAL_ALL_DATA.csv'
df = pd.read_csv(csv_path)

# Sınıf Dağılımını Hatırlayalım
print("\n--- EĞİTİLECEK VERİ SETİ ---")
print(df['label'].value_counts().sort_index())

# Class Weights (Sınıf Ağırlıkları) Hesaplama
# Az olan sınıfa yüksek ağırlık vereceğiz.
class_counts = df['label'].value_counts().sort_index()
total_samples = len(df)
num_classes = len(class_counts)

# Formül: Toplam / (Sınıf Sayısı * Sınıfın Frekansı)
class_weights = total_samples / (num_classes * class_counts)

print("\n--- HESAPLANAN SINIF AĞIRLIKLARI (Class Weights) ---")
print("(Model bu katsayıları kullanarak azınlık sınıflarına odaklanacak)")
print(class_weights)

# Bu ağırlıkları tensöre çevirip saklayalım, eğitimde kullanacağız.
weights_tensor = torch.tensor(class_weights.values, dtype=torch.float).to(device)

Kullanılan Cihaz: cuda

--- EĞİTİLECEK VERİ SETİ ---
label
1    53615
2      243
4       43
5     1215
Name: count, dtype: int64

--- HESAPLANAN SINIF AĞIRLIKLARI (Class Weights) ---
(Model bu katsayıları kullanarak azınlık sınıflarına odaklanacak)
label
1      0.256999
2     56.703704
4    320.441860
5     11.340741
Name: count, dtype: float64


In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
import torch

# --- 1. ETİKET HARİTALAMA (MAPPING) ---
# PyTorch sınıf indekslerinin 0'dan başlamasını ister.
# Mevcut: 1, 2, 4, 5 -> Hedef: 0, 1, 2, 3
label_map = {
    1: 0, # Normal
    2: 1, # İyi Huylu
    4: 2, # Şüpheli (Azınlık)
    5: 3  # Kanser
}

# Veri setine yeni 'target' sütununu ekleyelim
df['target'] = df['label'].map(label_map)

print("Etiketler PyTorch formatına (0-3) dönüştürüldü.")

# --- 2. KATMANLI BÖLME (STRATIFIED SPLIT) ---
# Önce Train (%80) ve Kalan (%20) olarak bölüyoruz
# stratify=df['target'] sayesinde her sınıftan eşit oranda alıyor.
train_df, temp_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['target'],
    random_state=42
)

# Kalan %20'yi de Validation (%10) ve Test (%10) olarak ikiye bölüyoruz
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,
    stratify=temp_df['target'],
    random_state=42
)

# --- 3. KAYIT (SAVE) ---
# Bölünen dosyaları Drive'a kaydediyoruz ki sabit kalsınlar.
save_dir = '/content/drive/MyDrive/SPLIT_DATA'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

train_df.to_csv(f'{save_dir}/train.csv', index=False)
val_df.to_csv(f'{save_dir}/val.csv', index=False)
test_df.to_csv(f'{save_dir}/test.csv', index=False)

print("\n--- BÖLME İŞLEMİ TAMAMLANDI VE KAYDEDİLDİ ---")
print(f"Kayıt Yeri: {save_dir}")
print(f"Eğitim (Train) Seti  : {len(train_df)} veri")
print(f"Doğrulama (Val) Seti : {len(val_df)} veri")
print(f"Test Seti            : {len(test_df)} veri")

# Kritik Kontrol: Sınıf 4 (En az olan) her sete dağılmış mı?
print("\n--- KRİTİK KONTROL: Sınıf 4 (Şüpheli) Dağılımı ---")
print(f"Train içinde Sınıf 4 Sayısı: {len(train_df[train_df['label']==4])}")
print(f"Val   içinde Sınıf 4 Sayısı: {len(val_df[val_df['label']==4])}")
print(f"Test  içinde Sınıf 4 Sayısı: {len(test_df[test_df['label']==4])}")

Etiketler PyTorch formatına (0-3) dönüştürüldü.

--- BÖLME İŞLEMİ TAMAMLANDI VE KAYDEDİLDİ ---
Kayıt Yeri: /content/drive/MyDrive/SPLIT_DATA
Eğitim (Train) Seti  : 44092 veri
Doğrulama (Val) Seti : 5512 veri
Test Seti            : 5512 veri

--- KRİTİK KONTROL: Sınıf 4 (Şüpheli) Dağılımı ---
Train içinde Sınıf 4 Sayısı: 35
Val   içinde Sınıf 4 Sayısı: 4
Test  içinde Sınıf 4 Sayısı: 4


In [7]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

# --- 1. DATASET SINIFI (Veri Okuyucu) ---
class BreastCancerDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        csv_file: 'train.csv', 'val.csv' veya 'test.csv' yolu
        transform: Uygulanacak görüntü dönüşümleri (Augmentation)
        """
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # 1. Resim yolunu al
        img_path = self.data.iloc[idx]['path']

        # 2. Etiketi al (0, 1, 2, 3)
        label = int(self.data.iloc[idx]['target'])

        # 3. Resmi aç (RGB formatında)
        try:
            image = Image.open(img_path).convert("RGB")
        except Exception as e:
            # Eğer resim bozuksa siyah bir resim oluştur (Kodun patlamaması için)
            print(f"Hata: Resim okunamadı {img_path}")
            image = Image.new('RGB', (224, 224), (0, 0, 0))

        # 4. Dönüşümleri uygula (Resize, Augmentation, Tensor)
        if self.transform:
            image = self.transform(image)

        return image, label

# --- 2. DÖNÜŞÜMLER (TRANSFORMS) ---
# Modelin gözü: Resimleri nasıl görecek?

# Eğitim Dönüşümleri (Ağır Augmentation - Veri Çoğaltma)
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),        # Boyut sabitleme
    transforms.RandomHorizontalFlip(p=0.5), # %50 ihtimalle yatay çevir
    transforms.RandomVerticalFlip(p=0.5),   # %50 ihtimalle dikey çevir
    transforms.RandomRotation(15),        # 15 derece döndür
    transforms.ColorJitter(brightness=0.2, contrast=0.2), # Işıkla oyna
    transforms.ToTensor(),                # Sayısal Tensöre çevir
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Renk normalizasyonu
])

# Test/Validation Dönüşümleri (Sade - Sadece Boyutlandırma)
val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- 3. TEST (Kod çalışıyor mu?) ---
# Sadece deneme amaçlı train setinden bir parça yüklüyoruz
try:
    test_ds = BreastCancerDataset(
        csv_file='/content/drive/MyDrive/SPLIT_DATA/train.csv',
        transform=train_transforms
    )

    img, lbl = test_ds[0] # İlk resmi al
    print("--- DATASET TEST RAPORU ---")
    print(f"Örnek Resim Boyutu (Tensor): {img.shape}")
    print(f"Örnek Etiket: {lbl}")
    print("Sistem başarıyla kuruldu. Resimler okunabiliyor.")

except Exception as e:
    print(f"BİR HATA OLUŞTU: {e}")

--- DATASET TEST RAPORU ---
Örnek Resim Boyutu (Tensor): torch.Size([3, 224, 224])
Örnek Etiket: 0
Sistem başarıyla kuruldu. Resimler okunabiliyor.


In [8]:
from torch.utils.data import WeightedRandomSampler, DataLoader
import numpy as np

# --- 1. AĞIRLIKLI ÖRNEKLEYİCİ (WEIGHTED SAMPLER) HAZIRLIĞI ---

# Train setindeki tüm etiketleri bir listeye alalım
train_targets = train_df['target'].tolist()

# Daha önce hesapladığımız sınıf ağırlıklarını (class_weights) hatırlayalım
# label 1 -> 0.25 (Çok var, az al)
# label 4 -> 320.4 (Çok az, çok al)
# Dikkat: class_weights Series objesiydi, bunu bir sözlüğe (dict) çevirelim ki erişimi kolay olsun.
# label_map ile dönüşüm yapmıştık (1->0, 2->1, 4->2, 5->3). Ağırlıkları da buna uyduralım.

# Orijinal etiketlere göre ağırlıklar:
weight_dict = class_weights.to_dict()

# PyTorch etiketlerine (0,1,2,3) göre ağırlık haritası:
# label_map'in tersini kullanacağız veya manuel eşleştireceğiz.
# label_map = {1:0, 2:1, 4:2, 5:3}
mapped_weights = {
    0: weight_dict[1],   # Class 1 (Normal)
    1: weight_dict[2],   # Class 2 (İyi Huylu)
    2: weight_dict[4],   # Class 4 (Şüpheli - En Yüksek Ağırlık)
    3: weight_dict[5]    # Class 5 (Kanser)
}

print("PyTorch Sınıf Ağırlıkları:", mapped_weights)

# Her bir eğitim örneği için (44092 tane) bir ağırlık değeri atayacağız
samples_weights = [mapped_weights[t] for t in train_targets]

# Sampler Objesini Oluştur
# replacement=True: Bir resim aynı epochta birden fazla kez çekilebilir (Veri çoğaltma etkisi yaratır)
sampler = WeightedRandomSampler(
    weights=samples_weights,
    num_samples=len(samples_weights),
    replacement=True
)

# --- 2. DATALOADERS (VERİ YÜKLEYİCİLER) ---

# Batch Size: GPU belleğine göre ayarlanır. T4 için 32 veya 64 uygundur.
BATCH_SIZE = 32

# Train Loader (Sampler ile)
# shuffle=False olmak ZORUNDA çünkü sampler zaten karıştırıyor.
train_dataset = BreastCancerDataset(f'{save_dir}/train.csv', transform=train_transforms)
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler,  # Kritik nokta burası!
    num_workers=2
)

# Val Loader (Sıradan, karıştırmaya gerek yok)
val_dataset = BreastCancerDataset(f'{save_dir}/val.csv', transform=val_transforms)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

# Test Loader
test_dataset = BreastCancerDataset(f'{save_dir}/test.csv', transform=val_transforms)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2
)

print("\n--- DATALOADER HAZIR ---")
print(f"Train Loader: {len(train_loader)} batch (Her batch'te {BATCH_SIZE} resim)")
print("Sınıf 4 artık eğitim sırasında diğerleriyle eşit oranda görünecek.")

PyTorch Sınıf Ağırlıkları: {0: 0.256998974167677, 1: 56.7037037037037, 2: 320.4418604651163, 3: 11.34074074074074}

--- DATALOADER HAZIR ---
Train Loader: 1378 batch (Her batch'te 32 resim)
Sınıf 4 artık eğitim sırasında diğerleriyle eşit oranda görünecek.


**RESNET-50**

In [11]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

# --- 1. MODEL MİMARİSİ (ResNet50) ---
print("ResNet50 modeli indiriliyor ve yapılandırılıyor...")

# Önceden eğitilmiş ağırlıkları kullan
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

# Modelin son katmanını değiştir (4 Sınıf için)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Modeli GPU'ya taşı
model = model.to(device)

print(f"Model GPU'ya taşındı: {next(model.parameters()).device}")

# --- 2. HATA FONKSİYONU (LOSS FUNCTION) ---
# Sınıf ağırlıklarını (weights_tensor) kullanıyoruz
criterion = nn.CrossEntropyLoss(weight=weights_tensor)

# --- 3. OPTİMİZASYON (OPTIMIZER) ---
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# --- 4. ÖĞRENME HIZI PLANLAYICI (SCHEDULER) ---
# verbose parametresini kaldırdık
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.1,
    patience=3
)

print("Model, Loss Fonksiyonu ve Optimizer başarıyla hazırlandı.")

ResNet50 modeli indiriliyor ve yapılandırılıyor...
Model GPU'ya taşındı: cuda:0
Model, Loss Fonksiyonu ve Optimizer başarıyla hazırlandı.


In [12]:
import time
import copy
from tqdm import tqdm

# --- AYARLAR ---
EPOCHS = 15  # 15 tur dönecek (İstersen artırabilirsin)
best_val_loss = float('inf') # En iyi kaybı takip etmek için
save_path = '/content/drive/MyDrive/best_breast_cancer_model.pth'

history = {
    'train_loss': [], 'train_acc': [],
    'val_loss': [], 'val_acc': []
}

print(f"Eğitim başlıyor... Hedef: {EPOCHS} Epoch")
print("-" * 60)

start_time = time.time()

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    # --- 1. EĞİTİM MODU (TRAIN) ---
    model.train()
    running_loss = 0.0
    running_corrects = 0

    # Tqdm ile ilerleme çubuğu
    for inputs, labels in tqdm(train_loader, desc="Eğitiliyor"):
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Sıfırla -> İleri -> Hata Hesapla -> Geri -> Güncelle
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels) # Ağırlıklı Loss burada devreye giriyor

        loss.backward()
        optimizer.step()

        # İstatistikler
        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() / len(train_dataset)

    # --- 2. DOĞRULAMA MODU (VALIDATION) ---
    model.eval()
    val_loss = 0.0
    val_corrects = 0

    with torch.no_grad(): # Validation'da gradyan hesaplanmaz (Hız ve Hafıza)
        for inputs, labels in tqdm(val_loader, desc="Doğrulanıyor"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)

    val_epoch_loss = val_loss / len(val_dataset)
    val_epoch_acc = val_corrects.double() / len(val_dataset)

    # --- 3. RAPORLAMA VE KAYIT ---
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")
    print(f"Val   Loss: {val_epoch_loss:.4f} Acc: {val_epoch_acc:.4f}")

    # Geçmişi kaydet
    history['train_loss'].append(epoch_loss)
    history['train_acc'].append(epoch_acc.item())
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc.item())

    # Scheduler Adımı (Validation Loss düşmüyorsa öğrenme hızını kıs)
    scheduler.step(val_epoch_loss)

    # En iyi modeli kaydet (Checkpoint)
    if val_epoch_loss < best_val_loss:
        print(f"✔ İyileşme var! ({best_val_loss:.4f} -> {val_epoch_loss:.4f}). Model kaydediliyor...")
        best_val_loss = val_epoch_loss
        torch.save(model.state_dict(), save_path)
    else:
        print("Model bu turda gelişmedi.")

# --- BİTİŞ ---
total_time = time.time() - start_time
print("-" * 60)
print(f"Eğitim tamamlandı. Toplam Süre: {total_time // 60:.0f}m {total_time % 60:.0f}s")
print(f"En iyi Validation Loss: {best_val_loss:.4f}")
print(f"Model Kayıt Yeri: {save_path}")

Eğitim başlıyor... Hedef: 15 Epoch
------------------------------------------------------------

Epoch 1/15


Eğitiliyor:   0%|          | 0/1378 [00:00<?, ?it/s]

Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996406Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50993868

Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22670324
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50997678
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996800
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/30011530
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50993976
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/24065407
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/20587346
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/24055483
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/30011530
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22671003
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996352
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50997304
Hata: 

Eğitiliyor:   0%|          | 1/1378 [00:01<24:11,  1.05s/it]

Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22678787
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/51048972
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996083Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/24065530

Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/30011824
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22579870


...

Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50997488
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/30011553
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50994535
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996881
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/20588216
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996352
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50993787
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50998413
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/24065680
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50997796
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50994841
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/24055600
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/20588216
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22613796
Hata: 

Eğitiliyor:   6%|▌         | 76/1378 [00:11<03:01,  7.16it/s]

Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22670324
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/53580611
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996800
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/20587466
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/30011824
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/24065530Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/20587544

Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50994841
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50993976
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996352
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50996352
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/50993841
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22670978
Hata: Resim okunamadı /content/drive/MyDrive/INbreast_Cropped/22580732
Hata: 

Eğitiliyor:   6%|▌         | 77/1378 [00:11<03:13,  6.74it/s]


KeyboardInterrupt: 